In [71]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from alphavantage import *
from yahoo_query import *
from option_slam_earnings import *
from reuters_query import reuters_query, reuters_insiders
from alphaquery import alphaquery

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Options')
from optionsFunctions import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Earnings')
from yahoo_earnings import *
from lookup_earnings import lookup_earnings

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Processing')
from hv_calc import *

os.chdir(main_dir)

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir(dbs_dir)

earn_engine = create_engine('sqlite:///earningsHistory.db', echo=False)
yahoo_engine = create_engine('sqlite:///yahoo.db', echo = False)
reuters_engine = create_engine('sqlite:///reuters.db', echo=False)

In [72]:
tickers = ['CMC', 'AZZ', 'BBBY', 'MSM', 'LEN', 'GBX']

annual_statements = {}
quarterly_statements = {}
profiles = []
query = 'SELECT * FROM {} WHERE Underlying IN ' + str(tuple(tickers))

for yahoo_table in yahoo_engine.table_names():
    if 'BalanceSheet' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['balanceSheet'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['balanceSheet'] = curr_table
            
    elif 'IncomeStatement' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['incomeStatements'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['incomeStatements'] = curr_table
            
    elif 'CashFlow' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['cashFlows'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['cashFlows'] = curr_table
    elif 'profiles' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates()[['industry', 'sector']]
        profiles.append(curr_table)
    else:
        continue
profiles = pd.concat(profiles, axis = 0)

In [76]:
annual_statements['cashFlows'][annual_statements['cashFlows'].index == 'AZZ'].T

Underlying,AZZ,AZZ,AZZ,AZZ
year,2015-02-28 00:00:00,2016-02-29 00:00:00,2017-02-28 00:00:00,2018-02-28 00:00:00
capitalExpenditures,-29377000,-39861000,-41434000,-29612000
changeInCash,-5038000,17664000,-28889000,9551000
changeToAccountReceivables,"-23,205,000.0000","6,433,000.0000","8,680,000.0000","4,476,000.0000"
changeToInventory,-3363000,-2052000,-13754000,-9927000
changeToLiabilities,11025000,-2236000,1245000,1540000
changeToNetincome,20886000,5385000,14310000,90000
changeToOperatingActivities,-322000,11751000,-12188000,-13560000
depreciation,46089000,47417000,50357000,50526000
dividendsPaid,-14897000,-15482000,-16645000,-17678000


In [77]:
quarterly_statements['cashFlows'][quarterly_statements['cashFlows'].index == 'AZZ'].T

Underlying,AZZ,AZZ,AZZ,AZZ
quarter,2017-11-30 00:00:00,2018-02-28 00:00:00,2018-05-31 00:00:00,2018-08-31 00:00:00
capitalExpenditures,"-4,897,000.0000","-8,079,000.0000","-2,847,000.0000","-4,332,000.0000"
changeInCash,"7,102,000.0000","10,202,000.0000","-6,915,000.0000","-4,734,000.0000"
changeToAccountReceivables,"2,400,000.0000","15,334,000.0000","-50,015,000.0000","17,558,000.0000"
changeToInventory,"1,560,000.0000","1,817,000.0000","-1,362,000.0000","-809,000.0000"
changeToLiabilities,903000,7407000,5576000,-15601000
changeToNetincome,"14,122,000.0000","-18,883,000.0000","4,024,000.0000","2,655,000.0000"
changeToOperatingActivities,"4,103,000.0000","-1,794,000.0000","484,000.0000","1,954,000.0000"
depreciation,"12,856,000.0000","12,686,000.0000","13,071,000.0000","12,627,000.0000"
dividendsPaid,"-4,421,000.0000","-4,412,000.0000","-4,418,000.0000","-4,426,000.0000"


In [73]:
for key,val in annual_statements.items():
    print(key)
    print(val.columns)

balanceSheet
Index(['year', 'accountsPayable', 'capitalSurplus', 'cash', 'commonStock',
       'deferredLongTermAssetCharges', 'deferredLongTermLiab', 'goodWill',
       'intangibleAssets', 'inventory', 'longTermDebt', 'longTermInvestments',
       'maxAge', 'minorityInterest', 'netReceivables', 'netTangibleAssets',
       'otherAssets', 'otherCurrentAssets', 'otherCurrentLiab', 'otherLiab',
       'otherStockholderEquity', 'propertyPlantEquipment', 'retainedEarnings',
       'shortLongTermDebt', 'shortTermInvestments', 'totalAssets',
       'totalCurrentAssets', 'totalCurrentLiabilities', 'totalLiab',
       'totalStockholderEquity', 'treasuryStock'],
      dtype='object')
cashFlows
Index(['year', 'capitalExpenditures', 'changeInCash',
       'changeToAccountReceivables', 'changeToInventory',
       'changeToLiabilities', 'changeToNetincome',
       'changeToOperatingActivities', 'depreciation', 'dividendsPaid',
       'effectOfExchangeRate', 'investments', 'issuanceOfStock',
       '